**<h2>Importing Libraries**

In [40]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


**<h2>Using beautifulsoup for scraping**

In [0]:
url=('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
page=requests.get(url)

soup=BeautifulSoup(page.content,'html.parser')

**<h2>Getting the table**

We observe that the lines of html code containing the table have tags like 'table' with the class name 'wikitable sortable'.We will use this information to scrape the table first.


In [42]:
table=soup.find('table',class_='wikitable sortable')
print(table)

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

**<h2>Getting the rows**

Next,we can notice that a new row starts with the tag 'tr' and the column entry is through 'td'. We use this to scrape the postalcodes , boroughlist and neighbourhood list. 

In [0]:
postalCodeList = []
boroughList = []
neighbourhoodList = []

In [0]:
for row in table.find_all('tr'):
  cells=row.find_all('td')
  if(len(cells)>0):#as we are skipping the header
    postalCodeList.append(cells[0].text.rstrip('\n')) #the rstrip helps avoid the newline charachter
    boroughList.append(cells[1].text.rstrip('\n'))
    neighbourhoodList.append(cells[2].text.rstrip('\n'))


In [45]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighbourhood": neighbourhoodList})

toronto_df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**<h2> Dropping values**

We'll be dropping of the Borough Values which aren't assigned.

In [46]:
toronto_df.drop(toronto_df[toronto_df['Borough']=='Not assigned'].index,axis=0,inplace=True)
toronto_df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**<h2>Grouping Data**

Next we will be grouping the data on the basis of same Postal codes.

In [47]:
toronto_df=toronto_df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**<h2>Removing the same 'Not assigned from Neighbourhood'**



In [48]:
x=list(toronto_df['Neighbourhood'])
y=list(toronto_df['Borough'])
for i in range(len(x)):
  if(x[i]=='Not assigned'):
    x[i]=y[i]
toronto_df['Neighbourhood']=x
toronto_df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [49]:
toronto_df.shape

(103, 3)

**<h2>Adding Location Data**

Uploading the data.

In [50]:
from google.colab import files
uploaded = files.upload()

Saving Geospatial_Coordinates.csv to Geospatial_Coordinates (1).csv


In [51]:
import io
df = pd.read_csv(io.StringIO(uploaded['Geospatial_Coordinates.csv'].decode('utf-8')))
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Renaming the postal code column so as to perform merge function**

In [52]:
df.rename(columns={"Postal Code":"PostalCode"},inplace=True)
df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**<h2>Merging**

We will be merging the two data frames : toronto_df and df.

In [53]:
new_data=pd.merge(toronto_df,df)
new_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**<h2>Exploration**

Next, we'll be exploring the data we have been given and might, add to it.
First,we'll be visualizing the data we processed so far on the map using the folium library.
We, first, would need Toronto's geographical coordinates.

In [54]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tornoto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tornoto are 43.6534817, -79.3839347.


**<h3> Visualizing the Data obtained using Folium**

In [55]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(new_data['Latitude'], new_data['Longitude'], new_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [56]:
CLIENT_ID = 'PEWXJBPBDXD5EJ1JZJ31A31YA4041H5PTNAN3I5BDKTWNQVA' # your Foursquare ID
CLIENT_SECRET = 'ROIDG41LFNNCSLWDH1BTYM5GFVLBWTDIIKQA54HRWJJ4Z5IB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PEWXJBPBDXD5EJ1JZJ31A31YA4041H5PTNAN3I5BDKTWNQVA
CLIENT_SECRET:ROIDG41LFNNCSLWDH1BTYM5GFVLBWTDIIKQA54HRWJJ4Z5IB


In [57]:
new_data.loc[0, 'Neighbourhood']

'Malvern, Rouge'

We need to focus on the groups of East , West , Central and Downtown Toronto.
We'll be taking out that part of the data from the given dataset in the next few lines of code.

In [58]:
borough_names = list(new_data['Borough'].unique())

borough_with_toronto = []

for x in borough_names:
    if ("toronto" in x.lower()):
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [59]:
new_df=new_data[new_data['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
new_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [60]:
new_df.shape

(39, 5)

**<h3>Visualizing the new data**

In [61]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng,neigh,borough in zip(new_df['Latitude'], new_df['Longitude'], new_df['Neighbourhood'],new_df):
    label = neigh+borough
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

We noticed that the current data has just 39 enteries , which is too less to form any conclusions.Hence,we'll be using foursquare to get enteries within 500m of radius

In [0]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighbourhood in zip(new_df['Latitude'], new_df['Longitude'],new_df['PostalCode'],new_df['Borough'],new_df['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighbourhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

We'll now form the new dataframe with the new venues we got.

In [63]:
venue_df = pd.DataFrame(venues)
venue_df.columns=['PostalCode','Borough','Neighbourhood','BLatitude','BLongitude','Vname','Vlat','Vlong','Vcategory']
venue_df.head()

,PostalCode,Borough,Neighbourhood,BLatitude,BLongitude,Vname,Vlat,Vlong,Vcategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [68]:
venue_df.groupby(['PostalCode','Neighbourhood','Borough']).count()

,,,BLatitude,BLongitude,Vname,Vlat,Vlong,Vcategory
PostalCode,Neighbourhood,Borough,,,,,,
M4E,The Beaches,East Toronto,4,4,4,4,4,4
M4K,"The Danforth West, Riverdale",East Toronto,43,43,43,43,43,43
M4L,"India Bazaar, The Beaches West",East Toronto,22,22,22,22,22,22
M4M,Studio District,East Toronto,40,40,40,40,40,40
M4N,Lawrence Park,Central Toronto,3,3,3,3,3,3
M4P,Davisville North,Central Toronto,7,7,7,7,7,7
M4R,"North Toronto West, Lawrence Park",Central Toronto,23,23,23,23,23,23
M4S,Davisville,Central Toronto,32,32,32,32,32,32
M4T,"Moore Park, Summerhill East",Central Toronto,4,4,4,4,4,4


In the next line of code , we'll check the unique categories in the venues

In [70]:
print('There are {} uniques categories.'.format(len(venue_df['Vcategory'].unique())))

There are 242 uniques categories.


**<h2>Clustering**

We'll now form clusters with the data we have. We'll first create dummy variables for the process , followed by a combined dataframe for the final procedure. After that we'll implement Kmeans algorithm and try to comprehend our findings.

**<h3>One hot encoding the data**

In [72]:
# one hot encoding
toronto_onehot = pd.get_dummies(venue_df[['Vcategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = venue_df['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,

In [73]:
toronto_onehot.shape

(1618, 243)

We'll next get a mean of activities in each neighbourhood.

In [95]:
toronto_group = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_group.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,0.018182,0.000000,0.0,0.0,0.0,0.000000,0.018182,0.0,0.018182,0.036364,0.0,0.000000,0.0,0.018182,0.018182,0.0,0.036364,0.0,0.0,0.018182,0.0,0.0000,0.0,0.0,0.018182,0.000000,0.00000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.036364,0.0,0.0,0.0,0.0,0.036364,0.000000,0.0,0.0,0.000000,0.018182,0.054545,0.072727,0.0,0.0,0.0,0.0,0.0,0.0,0.018182,0.000000,0.018182,0.000000,0.0,0.0,0.0,0.018182,0.0,0.0,0.0,0.0,0.018182,0.000000,0.018182,0.000000,0.0,0.0,0.0,0.000000,

**Sorting Data**

The next few lines of code will narrow down the top 10 venues visited near each neighbourhood and will put it in a new dataframe.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
    

In [96]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_group['Neighbourhood']

for ind in np.arange(toronto_group.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_group.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Restaurant,Bakery,Cheese Shop,Seafood Restaurant,Beer Bar,Fish Market,Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Bakery,Coffee Shop,Grocery Store,Furniture / Home Store,Burrito Place,Stadium,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Park,Garden Center,Farmers Market,Fast Food Restaurant,Skate Park,Burrito Place,Butcher,Brewery,Auto Workshop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Plane
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Department Store,Salad Place,Burger Joint,Ramen Restaurant


**Performing Kmeans**

We'll be making 5 clusters.

In [100]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_group.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32)

In [102]:
toronto_merged.columns

Index(['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

In [103]:
neighbourhoods_venues_sorted.columns

Index(['Cluster Labels', 'Neighbourhood', '1st Most Common Venue',
       '2nd Most Common Venue', '3rd Most Common Venue',
       '4th Most Common Venue', '5th Most Common Venue',
       '6th Most Common Venue', '7th Most Common Venue',
       '8th Most Common Venue', '9th Most Common Venue',
       '10th Most Common Venue'],
      dtype='object')

**<h3>Adding the Cluster Labels**

In [106]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = new_data
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = pd.merge(neighbourhoods_venues_sorted, toronto_merged)

toronto_merged.head() # check the last columns!

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
0,1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Restaurant,Bakery,Cheese Shop,Seafood Restaurant,Beer Bar,Fish Market,Pub,M5E,Downtown Toronto,43.644771,-79.373306
1,1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Bakery,Coffee Shop,Grocery Store,Furniture / Home Store,Burrito Place,Stadium,Italian Restaurant,M6K,West Toronto,43.636847,-79.428191
2,1,"Business reply mail Processing Centre, South C...",Light Rail Station,Park,Garden Center,Farmers Market,Fast Food Restaurant,Skate Park,Burrito Place,Butcher,Brewery,Auto Workshop,M7Y,East Toronto,43.662744,-79.321558
3,1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Plane,M5V,Downtown Toronto,43.628947,-79.394420
4,1,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Department Store,Salad Place,Burger Joint,Ramen Restaurant,M5G,Downtown Toronto,43.657952,-79.387383


**Visualizing Our Clusters**

In [107]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'],toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**<h2>Examining Our Clusters**

**<h3>Cluster 1**

In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
5,Christie,Nightclub,Coffee Shop,Candy Store,Diner,Baby Store,Restaurant,Athletics & Sports,M6G,Downtown Toronto,43.669542,-79.422564
10,"Dufferin, Dovercourt Village",Music Venue,Bus Stop,Café,Middle Eastern Restaurant,Supermarket,Bar,Bank,M6H,West Toronto,43.669005,-79.442259
12,"Forest Hill North & West, Forest Hill Road Park",Sushi Restaurant,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,M5P,Central Toronto,43.696948,-79.411307
15,"High Park, The Junction South",Grocery Store,Furniture / Home Store,Fast Food Restaurant,Bookstore,Speakeasy,Cajun / Creole Restaurant,Flea Market,M6P,West Toronto,43.661608,-79.464763
18,Lawrence Park,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,M4N,Central Toronto,43.728020,-79.388790


**<h3>Cluster 2**

In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
0,Berczy Park,Restaurant,Bakery,Cheese Shop,Seafood Restaurant,Beer Bar,Fish Market,Pub,M5E,Downtown Toronto,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",Bakery,Coffee Shop,Grocery Store,Furniture / Home Store,Burrito Place,Stadium,Italian Restaurant,M6K,West Toronto,43.636847,-79.428191
2,"Business reply mail Processing Centre, South C...",Farmers Market,Fast Food Restaurant,Skate Park,Burrito Place,Butcher,Brewery,Auto Workshop,M7Y,East Toronto,43.662744,-79.321558
3,"CN Tower, King and Spadina, Railway Lands, Har...",Harbor / Marina,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Plane,M5V,Downtown Toronto,43.628947,-79.394420
4,Central Bay Street,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Department Store,Salad Place,Burger Joint,Ramen Restaurant,M5G,Downtown Toronto,43.657952,-79.387383
6,Church and Wellesley,Restaurant,Gay Bar,Café,Pub,Men's Store,Mediterranean Restaurant,Hotel,M4Y,Downtown Toronto,43.665860,-79.383160
7,"Commerce Court, Victoria Hotel",Hotel,American Restaurant,Gym,Japanese Restaurant,Deli / Bodega,Italian Restaurant,Seafood Restaurant,M5L,Downtown Toronto,43.648198,-79.379817
8,Davisville,Coffee Shop,Sushi Restaurant,Gym,Café,Italian Restaurant,Seafood Restaurant,Farmers Market,M4S,Central Toronto,43.704324,-79.388790
9,Davisville North,Hotel,Sandwich Place,Breakfast Spot,Park,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,M4P,Central Toronto,43.712751,-79.390197
11,"First Canadian Place, Underground city",Restaurant,Hotel,Gym,Seafood Restaurant,Asian Restaurant,Deli / Bodega,Salad Place,M5X,Downtown Toronto,43.648429,-79.382280


**<h3>Cluster 3**

In [110]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
35,The Beaches,Pub,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,M4E,East Toronto,43.676357,-79.293031


**<h3>Cluster 4**

In [111]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
27,Roselawn,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,M5N,Central Toronto,43.711695,-79.416936


**<h3>Cluster 5**

In [112]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
20,"Moore Park, Summerhill East",Summer Camp,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,M4T,Central Toronto,43.689574,-79.383160
26,Rosedale,Yoga Studio,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,M4W,Downtown Toronto,43.679563,-79.377529


**<h1>Conclusion**

Cluster 1 seems to be belonging to residential areas. Their most popular venues are all what should be near residential areas that is, cafes, yoga studios,bookstores,baby stores etc.They seem to be small towns.

Cluster 2 has the maximum number of places. From the looks of their most popular venues it seems like they are cities and tourist place mixed.It has the maximum number of restaurants and hotels as their popular places.Most of them are in Downtown Toronto.It must be a very happening place.

Cluster 3 and Cluster 4 are the beaches and roselawn,respectively.

Cluster 5 belongs to the neighbourhood of Moore Park,Summerhill East and Rosedale.